In [122]:
# loading packages
import pandas as pd
import numpy as np
import datetime
import sklearn
import matplotlib.pyplot as plt

from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import helper

#### getting list of different metros zips to split model if necessary 

In [11]:
# lists of relevent zipcodes
target_zips = pd.read_csv(r'C:\Users\robla\Desktop\nycdsa\Capstone\data_dump\city_zip.csv', index_col = 0)
target_zips.columns = ['metro','zip_code']
houston_zips_list = list(target_zips[target_zips['metro']=='houston']['zip_code'].unique())
paso_zips_list = list(target_zips[target_zips['metro']=='el_paso']['zip_code'].unique())
san_zips_list = list(target_zips[target_zips['metro']=='san_antonio']['zip_code'].unique())
austin_zips_list = list(target_zips[target_zips['metro']=='austin']['zip_code'].unique())
dallas_zips_list = list(target_zips[target_zips['metro']=='dfw']['zip_code'].unique())


#### Loading and processing data to put into model

In [12]:
# load data
texas_data = pd.read_csv(r'C:\Users\robla\Desktop\nycdsa\Capstone\Pipeline Data\merged_texas_data.csv', index_col = 0,
                   parse_dates = ['Time']
                  )

acs_data = pd.read_csv(r'C:\Users\robla\Desktop\nycdsa\Capstone\Pipeline Data\merged_acs_data.csv', index_col = 0,
                      parse_dates = ['Time'])

zri = pd.read_csv(r'C:\Users\robla\Desktop\nycdsa\Capstone\Pipeline Data\long_interpolated_target.csv', index_col = 0,
                   parse_dates=['Time']
                  )
#'new_feature'
# adding shift to zri
zri_shift = helper.time_lag_merge(zri, zri, {
    12:['zori_ssa',#'new_feature'
       ],
    13:['zori_ssa'],
    18:['zori_ssa'],
    24:['zori_ssa']
},
                                          return_full = True
                                         )

# there should now be extra values after our target. 
# We are gonna remove the missing values that happen at the start of our inputs tho
zri_shift = zri_shift.sort_values('Time')
zri_shift = zri_shift.dropna(subset = ['zori_ssa_24_month_shift'],axis='index',
                             how = 'any').reset_index(drop = True)
# Adding the shift values
zri_shift.loc[:,'zori_ssa_1_diff_lag_12'] = (zri_shift.loc[:,'zori_ssa_12_month_shift'] -
                                             zri_shift.loc[:,'zori_ssa_13_month_shift'])
zri_shift.loc[:,'zori_ssa_6_diff_lag_12'] = (zri_shift.loc[:,'zori_ssa_12_month_shift'] -
                                             zri_shift.loc[:,'zori_ssa_18_month_shift'])
zri_shift.loc[:,'zori_ssa_12_diff_lag_12'] = (zri_shift.loc[:,'zori_ssa_12_month_shift'] -
                                             zri_shift.loc[:,'zori_ssa_24_month_shift'])
zri_shift['zori_ssa_12_diff_lag_12_per'] = (zri_shift['zori_ssa_12_diff_lag_12']/
                                           zri_shift['zori_ssa_12_month_shift'])

zri_shift = zri_shift[['Time','zip_code','zori_ssa', #'new_feature'
                       'zori_ssa_12_month_shift',
                       'zori_ssa_1_diff_lag_12', 
                       'zori_ssa_6_diff_lag_12',
                       'zori_ssa_12_diff_lag_12_per'
                      ]]


# merge non acs data 
extra_shift = ['Gross Value Natural Gas Production', 'sap_case_shiller_index']
merged_df = helper.time_lag_merge(zri_shift, 
                                                    texas_data, {
    12:list(texas_data.drop(columns = ['Time','zip_code']+extra_shift
                            ).columns),
    13:extra_shift
},
                                          return_full = True
                                         )
# merge acs data
acs_1_cols = [
    'black_pop',
    'white_pop',
    'hispanic_pop',
    'high_school_diploma',
    'female_female_households',
    'armed_forces',
    'children',
    'black_pop_annual_pct_change',
    'white_pop_annual_pct_change',
    'hispanic_pop_annual_pct_change',
    'high_school_diploma_annual_pct_change',
    'children_annual_pct_change',
    ]
merged_df = helper.time_lag_merge(merged_df, 
                                                    acs_data, {
    36:list(acs_data.drop(columns = ['Time','zip_code'] + acs_1_cols).columns),
    48:acs_1_cols                                              
},
                                          return_full = True
                                         )
# # visualize missing values. it should be that acs 2 does not have a single zipcode
# # then removing that line and checking to see that there are no more missing values.
merged_df = merged_df.loc[merged_df['Time']>datetime.datetime(2016,6,2),:
                          ].reset_index(drop=True)
merged_df = merged_df.loc[merged_df['Time']<datetime.datetime(2022,7,2),:
                          ].reset_index(drop=True)
merged_df = merged_df.sort_values('Time')
merged_df = merged_df.dropna(subset = ['single_women_36_month_shift'],axis='index',
                             how = 'any').reset_index(drop = True)


#### Adding net approve feature.

In [13]:
merged_df['tx_net_approve_12_month_shift'] = (merged_df['tx_is_better_12_month_shift'] - 
                                              merged_df['tx_is_worse_12_month_shift'])


#### Features to put into model. Splitting test, train, forecast

In [14]:
# creating list of variables to put into the model. initialy is all non index and target
X_vals = [
    'zori_ssa_12_month_shift',
    'zori_ssa_1_diff_lag_12',
    'zori_ssa_6_diff_lag_12',
    'total_sales_tax_12_month_shift',
    'housing_units_over_50_units_36_month_shift',
    'housing_units_built_1960_to_1969_36_month_shift',
    'black_pop_48_month_shift',
    'zori_ssa_12_diff_lag_12_per',
    'children_annual_pct_change_48_month_shift',
    'female_40_to_44_annual_pct_change_36_month_shift',
    'housing_units_10_to_19_units_annual_pct_change_36_month_shift',
    'sales_tax_rate_annual_pct_change_12_month_shift',
    'female_female_households_48_month_shift',
    'women_with_associate_degree_annual_pct_change_36_month_shift',
    'average_household_size_owners_annual_pct_change_36_month_shift',
    'units_paying_cash_rent_annual_pct_change_36_month_shift',
    'quintile_1_upper_limit_annual_pct_change_36_month_shift',
    'Gross Value Natural Gas Production_13_month_shift',
    'women_with_doctoral_degree_annual_pct_change_36_month_shift',
    'total_sales_tax_annual_pct_change_12_month_shift',
    'housing_units_built_1940_to_1949_36_month_shift',
    'housing_units_built_1980_to_1989_annual_pct_change_36_month_shift',
    'female_35_to_39_annual_pct_change_36_month_shift',
    'bicycle_population_36_month_shift',
    'housing_units_20_to_49_units_annual_pct_change_36_month_shift',
    'taxpayer_count_12_month_shift',
    'housing_units_5_to_9_units_36_month_shift',
    'high_school_diploma_annual_pct_change_48_month_shift',
    'driving_alone_population_annual_pct_change_36_month_shift',
    'taxpayer_is_ratio_12_month_shift',
    'motorcycle_population_36_month_shift',
    'housing_units_single_family_attached_annual_pct_change_36_month_shift',
    'white_pop_annual_pct_change_48_month_shift',
    'taxpayer_cl_ratio_annual_pct_change_12_month_shift',
    'taxpayer_is_ratio_annual_pct_change_12_month_shift',
    'housing_units_built_1940_to_1949_annual_pct_change_36_month_shift',
    'black_pop_annual_pct_change_48_month_shift',
    'Gross Value Natural Gas Production_annual_pct_change_12_month_shift',
    'housing_units_single_family_attached_owned_36_month_shift',
    'single_women_annual_pct_change_36_month_shift',
    'housing_units_built_1930_to_1939_36_month_shift',
    'housing_units_built_1930_to_1939_annual_pct_change_36_month_shift',
    'female_25_to_29_annual_pct_change_36_month_shift',    
    'tx_net_approve_12_month_shift'
    
]
y_val = 'zori_ssa'

# split train and test based on a year in advance.
train = merged_df.loc[merged_df['Time']<datetime.datetime(2020,7,2),:].reset_index(drop=True)
post_train = merged_df.loc[merged_df['Time']>datetime.datetime(2020,7,2),:].reset_index(drop=True)
# test will have all zips. 
test = post_train.loc[post_train['Time']<datetime.datetime(2021,7,2),:].reset_index(drop=True)
forecast = post_train.loc[post_train['Time']>datetime.datetime(2021,7,2),:].reset_index(drop=True)

# set up x and y values with a scaler
# train first
scaler = StandardScaler(with_mean=False)
X = train[X_vals]
X = scaler.fit_transform(X)
y = train[y_val]
# test all
X_test = test[X_vals]
X_test = scaler.transform(X_test)
y_test = test[y_val]
# forecasted values
X_forecast = forecast[X_vals]
X_forecast = scaler.transform(X_forecast)


#### Construct model

In [17]:
lasso = Lasso(max_iter = 50000, random_state = 33)
alphas = [0.1,0.2,0.3, 0.6, 1]
tuned_parameters = [{'alpha': alphas}]
print(f'Performing Grid Search with alphas of: {alphas}')
clf = GridSearchCV(lasso, tuned_parameters, 
                    cv=5,n_jobs = -1, verbose=3,
                  scoring = 'neg_mean_squared_error')
# training on all non Houston zipcodes
clf.fit(X, y)

print(f"Best alpha {clf.best_params_['alpha']}")

Performing Grid Search with alphas of: [0.1, 0.2, 0.3, 0.6, 1]
Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best alpha 0.1


[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:    4.0s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    4.0s finished


In [16]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

#### Creating Coefficients of Model

In [18]:
coef_df = pd.DataFrame({'features':test[X_vals].columns,'coefs':clf.best_estimator_.coef_})
unselected_coef_df = coef_df[coef_df['coefs']==0]
coef_df = coef_df[coef_df['coefs']!=0]
coef_df['coefs_abs'] = abs(coef_df['coefs'])
coef_df = coef_df.sort_values('coefs_abs',ascending=False).reset_index(drop=True)
coef_df

,features,coefs,coefs_abs
0,zori_ssa_12_month_shift,237.244856,237.244856
1,zori_ssa_1_diff_lag_12,26.304015,26.304015
2,zori_ssa_6_diff_lag_12,-16.864855,16.864855
3,zori_ssa_12_diff_lag_12_per,4.323890,4.323890
4,total_sales_tax_12_month_shift,-4.170712,4.170712
5,housing_units_over_50_units_36_month_shift,-3.671122,3.671122
6,housing_units_built_1960_to_1969_36_month_shift,3.401850,3.401850
7,children_annual_pct_change_48_month_shift,-2.595727,2.595727
8,black_pop_48_month_shift,2.446128,2.446128
9,housing_units_single_family_attached_owned_36_...,-2.011038,2.011038


####  testing all zipcodes on the model

In [21]:
y_pred_test = clf.predict(X_test)

test.loc[:,'pred'] = y_pred_test
test.loc[:,'pred_difference'] = test.loc[:,y_val] - y_pred_test

rms = np.sqrt(mean_squared_error(y_test, y_pred_test))
rms


53.56478327318192

####  Feeding forecast values into model

In [22]:
y_pred_fore = clf.predict(X_forecast)
forecast.loc[:,'pred'] = y_pred_fore

#### Construct timeline of zori and what prediciton status they are.

In [23]:
# put actual data into correct format
zori_pred_act = zri[['Time','zip_code','zori_ssa']].dropna(subset=['zori_ssa'])
zori_pred_act['model_code'] = 'actual_values'
# put train into correct format
zori_pred_train = test[['Time','zip_code','pred']]
zori_pred_train.columns = ['Time','zip_code','zori_ssa']
zori_pred_train['model_code'] = 'lasso_base'# just code for the lasso model






# put forecast into correct format
zori_pred_fore = forecast[['Time','zip_code','pred']]
zori_pred_fore.columns = ['Time','zip_code','zori_ssa']
zori_pred_fore['model_code'] = 'lasso_base'
# concat them together
zori_pred = pd.concat([zori_pred_act, zori_pred_train, 
                       zori_pred_fore]).reset_index(drop=True)

zori_pred.to_csv('zori_pred_lasso_base.csv')

C:\Users\robla\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\robla\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


# Constructing confidence intervals for zori predictions and forecasted values

In [127]:
pred_and_fore

,Time,zip_code,zori_ssa,model_code
19565,2020-08-01,76135,1459.190909,lasso_base
19566,2020-08-01,76107,1498.295853,lasso_base
19567,2020-08-01,75062,1573.986251,lasso_base
19568,2020-08-01,78753,1262.419460,lasso_base
19569,2020-08-01,78744,1254.355824,lasso_base
19570,2020-08-01,75189,1723.615600,lasso_base
19571,2020-08-01,76036,1533.984979,lasso_base
19572,2020-08-01,76112,890.815340,lasso_base
19573,2020-08-01,78255,1301.750403,lasso_base
19574,2020-08-01,78748,1293.507251,lasso_base


In [133]:
coef_df

,features,coefs,coefs_abs
0,zori_ssa_12_month_shift,237.244856,237.244856
1,zori_ssa_1_diff_lag_12,26.304015,26.304015
2,zori_ssa_6_diff_lag_12,-16.864855,16.864855
3,zori_ssa_12_diff_lag_12_per,4.323890,4.323890
4,total_sales_tax_12_month_shift,-4.170712,4.170712
5,housing_units_over_50_units_36_month_shift,-3.671122,3.671122
6,housing_units_built_1960_to_1969_36_month_shift,3.401850,3.401850
7,children_annual_pct_change_48_month_shift,-2.595727,2.595727
8,black_pop_48_month_shift,2.446128,2.446128
9,housing_units_single_family_attached_owned_36_...,-2.011038,2.011038


In [147]:
len(avg_pred_and_fore) - 42

-18

In [174]:
pred_and_fore = zori_pred.loc[zori_pred['model_code'] == 'lasso_base']


import scipy.stats as st

avg_pred_and_fore = pred_and_fore[['Time','zip_code','zori_ssa']].groupby(by = 'Time').mean()
std_pred_and_fore = pred_and_fore[['Time','zip_code','zori_ssa']].groupby(by = 'Time').std()


avg_pred_and_fore.loc[:,'cumulative_avg'] = avg_pred_and_fore['zori_ssa'].expanding().mean()
avg_pred_and_fore.loc[:,'cumulative_std'] = avg_pred_and_fore['zori_ssa'].expanding().std()


avg_pred_and_fore.loc[:,'lower_prediction_bound'] =  avg_pred_and_fore['zori_ssa'] - 2*avg_pred_and_fore['cumulative_std']
avg_pred_and_fore.loc[:,'upper_prediction_bound'] =  avg_pred_and_fore['zori_ssa'] + 2*avg_pred_and_fore['cumulative_std']


avg_pred_and_fore.loc[:,'lower_prediction_bound_zip'] =  avg_pred_and_fore['zori_ssa'] - 2*std_pred_and_fore['zori_ssa']
avg_pred_and_fore.loc[:,'upper_prediction_bound_zip'] =  avg_pred_and_fore['zori_ssa'] + 2*std_pred_and_fore['zori_ssa']

avg_pred_and_fore[['zori_ssa','lower_prediction_bound','upper_prediction_bound','lower_prediction_bound_zip','upper_prediction_bound_zip']].reset_index()

,Time,zori_ssa,lower_prediction_bound,upper_prediction_bound,lower_prediction_bound_zip,upper_prediction_bound_zip
0,2020-08-01,1449.378023,NaN,NaN,975.475641,1923.280406
1,2020-09-01,1454.005958,1447.461070,1460.550846,979.708772,1928.303144
2,2020-10-01,1458.530523,1449.377829,1467.683218,983.441378,1933.619668
3,2020-11-01,1453.194497,1445.681068,1460.707926,981.611996,1924.776998
4,2020-12-01,1455.547139,1448.850519,1462.243758,984.190402,1926.903875
5,2021-01-01,1459.957610,1452.308628,1467.606592,986.707234,1933.207987
6,2021-02-01,1462.791964,1453.706539,1471.877389,991.099635,1934.484293
7,2021-03-01,1469.359489,1456.816437,1481.902540,997.815192,1940.903786
8,2021-04-01,1471.231097,1456.490224,1485.971969,999.373551,1943.088642
9,2021-05-01,1480.461713,1461.181674,1499.741753,1003.462737,1957.460690


In [175]:
std_pred_and_fore

,zip_code,zori_ssa
Time,,
2020-08-01,1429.326352,236.951191
2020-09-01,1429.326352,237.148593
2020-10-01,1429.326352,237.544573
2020-11-01,1429.326352,235.791250
2020-12-01,1429.326352,235.678368
2021-01-01,1429.326352,236.625188
2021-02-01,1429.326352,235.846165
2021-03-01,1429.326352,235.772148
2021-04-01,1429.326352,235.928773


In [184]:
df = avg_pred_and_fore[['zori_ssa','lower_prediction_bound','upper_prediction_bound','lower_prediction_bound_zip','upper_prediction_bound_zip']].reset_index()

df = pd.melt(df,
    id_vars=['Time'],
    value_vars=[
        'zori_ssa',
        'lower_prediction_bound',
        'upper_prediction_bound',
        'lower_prediction_bound_zip',
        'upper_prediction_bound_zip'
    ],
)

df.columns = ['Time','model_code','zori_ssa']
imp = {'zori_ssa':'lasso_pred',
       'lower_prediction_bound':'lower_prediction_bound',
       'upper_prediction_bound':'upper_prediction_bound',
       'lower_prediction_bound_zip':'lower_prediction_bound_zip',
       'upper_prediction_bound_zip':'upper_prediction_bound_zip'
       
      }
df.loc[:,'model_code'] = df['model_code'].apply(lambda x: imp[x])


import seaborn as sns
import plotly.express as px
px.line(data_frame = df, x = 'Time', y = 'zori_ssa', color = 'model_code',title = 'prediction confidence intervals')


In [182]:
df.to_csv(r'C:\Users\robla\Desktop\nycdsa\Capstone\Pipeline Data\confi_intervals.csv')

In [183]:
df

,Time,model_code,zori_ssa
0,2020-08-01,lasso_pred,1449.378023
1,2020-09-01,lasso_pred,1454.005958
2,2020-10-01,lasso_pred,1458.530523
3,2020-11-01,lasso_pred,1453.194497
4,2020-12-01,lasso_pred,1455.547139
5,2021-01-01,lasso_pred,1459.957610
6,2021-02-01,lasso_pred,1462.791964
7,2021-03-01,lasso_pred,1469.359489
8,2021-04-01,lasso_pred,1471.231097
9,2021-05-01,lasso_pred,1480.461713
